In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import string
from sklearn.model_selection import train_test_split
import gensim
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
import nltk as nt
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras.layers import Dropout, Activation, Dense
from keras.layers.normalization import batch_normalization
from sklearn.preprocessing import LabelEncoder

In [22]:
df = pd.read_excel("final_df.xlsx")

In [23]:
df.drop("Unnamed: 0",inplace=True,axis=1)

In [24]:
X = df["Review"]
y = df["Spoiler_flag"]

In [25]:
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=1)

In [26]:
X_train_2, X_val, y_train_2, y_val = train_test_split(X_train,y_train,test_size=0.2,random_state=1)

In [27]:
import gensim

def preprocess_text(text):
    if isinstance(text, float):
        text = str(text)
    return gensim.utils.simple_preprocess(text)



In [28]:
tokenize_train = X_train_2.apply(preprocess_text)

In [29]:
tokenize_test = X_val.apply(preprocess_text)

In [30]:
from nltk.corpus import stopwords

nt.download('stopwords')
sw = stopwords.words( 'english' )

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\noahr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
def remove_stopwords(text):
    l = []
    for i in text:
        if i not in sw:
            l.append(i.strip('\'\"'))
    return l

In [34]:
tokenize_train = tokenize_train.apply(remove_stopwords)


In [35]:
tokenize_test = tokenize_test.apply(remove_stopwords)

In [36]:
#Building Word2Vec Model
w2v_num_features = 512
w2v_model = gensim.models.Word2Vec(tokenize_train,size=w2v_num_features, window=150,min_count=2,sample=0.001)

In [37]:
def document_vectorizer(corpus,model,num_features):
  vocabulary=set(model.wv.index2entity)

  def average_word_vectors(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords=0
    for word in words:
      if word in vocabulary:
        nwords=nwords+1
        feature_vector=np.add(feature_vector, model.wv[word])
    if nwords:
      feature_vector = np.divide(feature_vector,nwords)

    return feature_vector
  features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features) for tokenized_sentence in corpus]

  return  np.array(features)

In [38]:
avg_wv_train_features = document_vectorizer(corpus=tokenize_train,model=w2v_model, num_features=w2v_num_features)


In [39]:
avg_wv_test_features = document_vectorizer(corpus=tokenize_test,model=w2v_model, num_features=w2v_num_features)


In [40]:
#Basic Neural network model

In [120]:
from keras.layers.normalization.batch_normalization_v1 import BatchNormalization
from keras.engine import input_layer
from keras.regularizers import l2
from keras import regularizers
from keras.layers import LeakyReLU


def construct_deepnn(num_input_features):
  dnn_model = Sequential()
  dnn_model.add(Dense(512, input_shape=(num_input_features,), kernel_initializer='glorot_uniform'))
  dnn_model.add(BatchNormalization())
  dnn_model.add(LeakyReLU(alpha=0.1))
  dnn_model.add(Dropout(0.4))

  dnn_model.add(Dense(512, kernel_initializer='glorot_uniform'))
  dnn_model.add(BatchNormalization())
  dnn_model.add(LeakyReLU(alpha=0.1))
  dnn_model.add(Dropout(0.4))

  dnn_model.add(Dense(512, kernel_initializer='glorot_uniform'))
  dnn_model.add(BatchNormalization())
  dnn_model.add(LeakyReLU(alpha=0.1))
  dnn_model.add(Dropout(0.4))
  

  dnn_model.add(Dense(1))
  dnn_model.add(Activation('sigmoid'))

  dnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  return dnn_model

In [121]:
w2v_dnn = construct_deepnn(num_input_features=w2v_num_features)

In [122]:
batch_size=100

w2v_dnn.fit(avg_wv_train_features, y_train_2, epochs=20, batch_size=batch_size, shuffle=True, validation_split=0.1, verbose=1)

Epoch 1/20
149/149 [==============================] - 2s 11ms/step - loss: 0.7040 - accuracy: 0.6239 - val_loss: 0.6127 - val_accuracy: 0.6687
Epoch 2/20
149/149 [==============================] - 1s 10ms/step - loss: 0.6388 - accuracy: 0.6525 - val_loss: 0.6006 - val_accuracy: 0.6917
Epoch 3/20
149/149 [==============================] - 1s 9ms/step - loss: 0.6168 - accuracy: 0.6618 - val_loss: 0.6198 - val_accuracy: 0.6772
Epoch 4/20
149/149 [==============================] - 1s 10ms/step - loss: 0.6075 - accuracy: 0.6720 - val_loss: 0.6035 - val_accuracy: 0.6911
Epoch 5/20
149/149 [==============================] - 1s 9ms/step - loss: 0.6009 - accuracy: 0.6777 - val_loss: 0.6028 - val_accuracy: 0.6760
Epoch 6/20
149/149 [==============================] - 1s 9ms/step - loss: 0.5941 - accuracy: 0.6855 - val_loss: 0.5959 - val_accuracy: 0.6875
Epoch 7/20
149/149 [==============================] - 1s 9ms/step - loss: 0.5901 - accuracy: 0.6892 - val_loss: 0.6100 - val_accuracy: 0.6784
Epo

In [123]:
w2v_dnn.save("deep_learning2.h5")

In [124]:
#LSTM

In [125]:
from collections import Counter

token_counter = Counter([token for review in tokenize_train for token in review])

vocab_map = {item[0]: index+1 for index,item in enumerate(dict(token_counter).items())}

In [126]:
max_index = np.max(list(vocab_map.values()))
vocab_map['PAD_INDEX'] = 0
vocab_map['NOT_FOUND_INDEX'] = max_index+1
vocab_size = len(vocab_map)

In [127]:
print('Vocabulary Size:', vocab_size)
print('Sample Size of Vocabulary Size:', dict(list(vocab_map.items())[10:20]))


Vocabulary Size: 56076
Sample Size of Vocabulary Size: {'jack': 11, 'nicholson': 12, 'young': 13, 'film': 14, 'making': 15, 'different': 16, 'exercise': 17, 'today': 18, 'putting': 19, 'show': 20}


In [128]:
max_len = np.max([len(review) for review in tokenize_train])

In [129]:
# Convert tokeized train to numeric vectors
train_X = [[vocab_map[token] for token in line] for line in tokenize_train]

In [130]:
from keras.preprocessing import sequence


In [131]:
train_X = sequence.pad_sequences(train_X,maxlen=max_len)


In [132]:
# Convert tokeized test to numeric vectors

test_X = [[vocab_map[token] if vocab_map.get(token) else vocab_map['NOT_FOUND_INDEX']for token in line] for line in tokenize_test]

In [133]:
test_X = sequence.pad_sequences(test_X,maxlen=max_len)


In [145]:
from keras.regularizers import l2

EMBEDDING_DIMENSION = 256
LSTM_DIM = 64
REGULARIZATION_RATE = 0.01

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=EMBEDDING_DIMENSION, input_length=max_len))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(LSTM_DIM, dropout=0.4, recurrent_dropout=0.4, kernel_regularizer=l2(REGULARIZATION_RATE)))
model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(REGULARIZATION_RATE)))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [146]:
model.fit(train_X,y_train_2,epochs=10, batch_size=100, shuffle=True, validation_split=0.1, verbose=1)


Epoch 1/10
149/149 [==============================] - 491s 3s/step - loss: 1.2008 - accuracy: 0.6073 - val_loss: 0.6178 - val_accuracy: 0.6966
Epoch 2/10
149/149 [==============================] - 487s 3s/step - loss: 0.5949 - accuracy: 0.7190 - val_loss: 0.6101 - val_accuracy: 0.6899
Epoch 3/10
149/149 [==============================] - 488s 3s/step - loss: 0.5005 - accuracy: 0.8003 - val_loss: 0.6627 - val_accuracy: 0.6426
Epoch 4/10
149/149 [==============================] - 487s 3s/step - loss: 0.4501 - accuracy: 0.8373 - val_loss: 0.6962 - val_accuracy: 0.6887
Epoch 5/10
 79/149 [==============>...............] - ETA: 3:43 - loss: 0.3742 - accuracy: 0.8796

KeyboardInterrupt: 